# Uczenie maszynowe
### Maciej Łuczak 100908
### 24-02-2016

## Użyte biblioteki

In [55]:
import pandas as pd
from sklearn import preprocessing, grid_search
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.grid_search import RandomizedSearchCV
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.cross_validation import train_test_split

## Wczytywanie danych

### wczytanie danych uczących:

wczytanie z pliku csv:

In [56]:
startData = pd.read_csv("all_summary.txt", sep=";", index_col=None, na_values=['NA','NaN','','nan'], low_memory=False,
                   keep_default_na = False)

Usunięcie zbędnych wartości res_name w danych uczących:

In [57]:
filtered = startData[~startData["res_name"].isin(
        ["DA", "DC", "DT", "DU", "DG", "DI", "UNK", "UNX", "UNL", "PR", "PD", "Y1", "EU", "N", "15P", "UQ", "PX4", "NAN"])]

In [58]:
filtered = filtered[filtered["res_name"].notnull()]

In [59]:
filtered = filtered[filtered.res_name != None]

zagwarantowanie unikalności pary "res_name","pdb_code":

In [60]:
distinct = filtered.drop_duplicates(subset=["res_name","pdb_code"])

Pozostawienie tylko klas o liczności większej od 5:

In [61]:
endData = distinct.groupby('res_name').filter(lambda x: len(x) >= 5)

### wczytanie danych z pogrupowanymi etykietami

In [62]:
grupedLab = pd.read_csv("grouped_res_name.txt", sep=",", index_col=None, na_values=['NA','NaN','','nan'], low_memory=False,
                   keep_default_na = False)
endDataG = endData
endDataG['res_name']=grupedLab['res_name_group']

### wczytanie danych testowych:

In [63]:
endDataT = pd.read_csv("test_data.txt", sep=",", index_col=None, na_values=['NA','NaN','','nan'], low_memory=False,
                   keep_default_na = False)

## Przygotowanie danych do użycia w klasyfikatorach

### Dane uczące:

Zakodowanie etykiet w postaci liczb:

In [64]:
le = preprocessing.LabelEncoder()
le.fit(endData[['res_name']])
endData[['res_name']] = le.transform(endData[['res_name']])

Usunięcie kolumn z wartościami NULL i nie będących zmiennoprzecinkowymi:

In [65]:
completeColumnEndData = endData.count()==len(endData)
completeColumnEndData = completeColumnEndData.values
completeColumnEndData = endData.iloc[:,completeColumnEndData]
X = completeColumnEndData.select_dtypes(['float64'])
y = completeColumnEndData['res_name']

Sprawdzenie kształtu danych

In [66]:
print type(X)
print X.shape
print type(y)
print y.shape

<class 'pandas.core.frame.DataFrame'>
(10767, 73)
<class 'pandas.core.series.Series'>
(10767L,)


### Dane z pogrupowanymi etykietami

In [67]:
leG = preprocessing.LabelEncoder()
leG.fit(endDataG[['res_name']])
endDataG[['res_name']] = leG.transform(endDataG[['res_name']])

In [68]:
completeColumnEndDataG = endDataG.count()==len(endDataG)
completeColumnEndDataG = completeColumnEndDataG.values
completeColumnEndDataG = endDataG.iloc[:,completeColumnEndDataG]
X_gruped = completeColumnEndDataG.select_dtypes(['float64'])
y_gruped = completeColumnEndDataG['res_name']

In [69]:
print type(X_gruped)
print X_gruped.shape
print type(y_gruped)
print y_gruped.shape

<class 'pandas.core.frame.DataFrame'>
(10767, 73)
<class 'pandas.core.series.Series'>
(10767L,)


### Dane testowe:

In [70]:
completeColumnEndDataT = endDataT.count()==len(endDataT)
completeColumnEndDataT = completeColumnEndDataT.values
completeColumnEndDataT = endDataT.iloc[:,completeColumnEndDataT]
completeColumnEndDataT = completeColumnEndDataT.select_dtypes(['float64'])
col_names = [name for name in completeColumnEndDataT if name in X]
X_test = completeColumnEndDataT[col_names]
X = X[col_names]
X_gruped = X_gruped[col_names]

In [71]:
print type(X_test)
print X_test.shape

<class 'pandas.core.frame.DataFrame'>
(18917, 64)


## Klasyfikator K-Nearest Neighbours

In [19]:
knn = KNeighborsClassifier()
k_range = range(1, 31)
param_grid = dict(n_neighbors=k_range)
knnResult = grid_search.GridSearchCV(knn, param_grid=param_grid, scoring = 'recall')

Dla danych uczących:

In [20]:
knnResult.fit(X,y)
knnResult.best_params_

{'n_neighbors': 10}

In [24]:
knnResult.best_score_

0.74319680505247521

Dla danych pogrupowanych:

In [21]:
knnResult.fit(X_gruped,y_gruped)
knnResult.best_params_

{'n_neighbors': 10}

In [25]:
knnResult.best_score_

0.74319680505247521

## Random Forest

In [22]:
rf = RandomForestClassifier()

### Metoda Grid Search:

z względu na czasochłoność obliczeń w tej metodzie zostały wzięte pod uwagę tylko parametr ilości drzew i miary

In [27]:
param_grid = {"n_estimators" : range(5, 31),
              "criterion" : ["gini","entropy"]
             }

#### Dla danych uczących:

In [28]:
rfGS = grid_search.GridSearchCV(rf, param_grid=param_grid, scoring = 'recall_macro')
rfGS.fit(X, y)
rfGS.best_params_

{'criterion': 'gini', 'n_estimators': 19}

In [32]:
rfGS.best_score_

0.011122093461982351

#### Dla danych pogrupowanych:

In [30]:
rfGS_G = grid_search.GridSearchCV(rf, param_grid=param_grid, scoring = 'recall_macro')
rfGS_G.fit(X_gruped, y_gruped)
rfGS_G.best_params_

{'criterion': 'gini', 'n_estimators': 13}

In [31]:
rfGS_G.best_score_

0.012298314320852615

### Optymalizacja metodą Random Search

W metodzie Random Search było możliwe wzięcie pod uwagę większej ilości parametrów, zostały dołożone głębokość drzewa i maksymalna ilość kolumn bran apod uwagę przy podziale.

In [33]:
param_grid = {"n_estimators" : range(10, 40),
              "criterion" : ["gini","entropy"],
              "max_depth" : range(4, 10),
              "max_features" : [0.05,0.1,0.2,"log2","auto",None]
             }

#### Dla danych uczących:

In [36]:
rfRS = RandomizedSearchCV(rf, param_grid, cv=10, scoring='recall_macro', n_iter=10, random_state=5)
rfRS.fit(X,y)

RandomizedSearchCV(cv=10, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params={}, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'n_estimators': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], 'max_features': [0.05, 0.1, 0.2, 'log2', 'auto', None], 'criterion': ['gini', 'entropy'], 'max_depth': [4, 5, 6, 7, 8, 9]},
          pre_dispatch='2*n_jobs', random_state=5, refit=True,
          scoring='recall_macro', verbose=0)

In [ ]:
rfRS.grid_scores_

[mean: 0.07899, std: 0.01671, params: {'n_estimators': 25, 'max_features': 'log2', 'criterion': 'entropy', 'max_depth': 7},
 mean: 0.16558, std: 0.03630, params: {'n_estimators': 24, 'max_features': None, 'criterion': 'entropy', 'max_depth': 5},
 mean: 0.14793, std: 0.03283, params: {'n_estimators': 31, 'max_features': 'auto', 'criterion': 'entropy', 'max_depth': 9},
 mean: 0.11116, std: 0.01995, params: {'n_estimators': 22, 'max_features': 'auto', 'criterion': 'gini', 'max_depth': 9},
 mean: 0.06186, std: 0.01250, params: {'n_estimators': 30, 'max_features': 0.05, 'criterion': 'gini', 'max_depth': 8},
 mean: 0.08375, std: 0.01923, params: {'n_estimators': 12, 'max_features': 0.05, 'criterion': 'entropy', 'max_depth': 9},
 mean: 0.03851, std: 0.00719, params: {'n_estimators': 34, 'max_features': 0.05, 'criterion': 'gini', 'max_depth': 5},
 mean: 0.04330, std: 0.00830, params: {'n_estimators': 31, 'max_features': 0.05, 'criterion': 'entropy', 'max_depth': 6},
 mean: 0.11875, std: 0.02583, params: {'n_estimators': 15, 'max_features': 'log2', 'criterion': 'entropy', 'max_depth': 9},
 mean: 0.32010, std: 0.04724, params: {'n_estimators': 28, 'max_features': None, 'criterion': 'gini', 'max_depth': 9}]

Zostało zaobserwowane że klasyfikator działa najlepiej dla wartości 'max_features': None oraz wysokiej wartości 'max_depth' w związku z czym dalej zastosowano metodę Grid Search na parametrze 'max_depth' z ustaloną wartością 'max_features': None i 'n_estimators': 31

In [ ]:
param_grid_1 = {"n_estimators" : [31],
              "criterion" : ["gini"],
              "max_depth" : [11,13,15,17,23,27],
              "max_features" : [None]
             }
rfGS_2 = grid_search.GridSearchCV(rf, param_grid_1, cv=10, scoring='recall_macro')
rfGS_2.fit(X,y)
rfGS_2.grid_scores_

[mean: 0.41333, std: 0.04395, params: {'n_estimators': 31, 'max_features': None, 'criterion': 'gini', 'max_depth': 11},
 mean: 0.49008, std: 0.05489, params: {'n_estimators': 31, 'max_features': None, 'criterion': 'gini', 'max_depth': 13},
 mean: 0.58532, std: 0.05372, params: {'n_estimators': 31, 'max_features': None, 'criterion': 'gini', 'max_depth': 15},
 mean: 0.64643, std: 0.03839, params: {'n_estimators': 31, 'max_features': None, 'criterion': 'gini', 'max_depth': 17},
 mean: 0.67589, std: 0.02389, params: {'n_estimators': 31, 'max_features': None, 'criterion': 'gini', 'max_depth': 23},
 mean: 0.66867, std: 0.01727, params: {'n_estimators': 31, 'max_features': None, 'criterion': 'gini', 'max_depth': 27}]

Najlepszy więc okazał się zestaw parametrów:
mean: 0.67589, std: 0.02389, params: {'n_estimators': 31, 'max_features': None, 'criterion': 'gini', 'max_depth': 23}

#### Dla danych pogrupowanych

In [ ]:
rfRS_G = RandomizedSearchCV(rf, param_grid, cv=10, scoring='recall_macro', n_iter=10, random_state=5)
rfRS_G.fit(X,y)

In [ ]:
rfRS_G.grid_scores_

## Zapisanie klasufikatora

In [72]:
knn_dump = KNeighborsClassifier(n_neighbors=10)
knn_dump.fit(X,y)
joblib.dump(knn_dump, 'knnKlasyfikator')

['knnKlasyfikator',
 'knnKlasyfikator_01.npy',
 'knnKlasyfikator_02.npy',
 'knnKlasyfikator_03.npy',
 'knnKlasyfikator_04.npy',
 'knnKlasyfikator_05.npy',
 'knnKlasyfikator_06.npy',
 'knnKlasyfikator_07.npy',
 'knnKlasyfikator_08.npy',
 'knnKlasyfikator_09.npy']

In [75]:
knnG_dump = KNeighborsClassifier(n_neighbors=10)
knnG_dump.fit(X_gruped,y_gruped)
joblib.dump(knnG_dump, 'knnKlasyfikator_gruped')

['knnKlasyfikator_gruped',
 'knnKlasyfikator_gruped_01.npy',
 'knnKlasyfikator_gruped_02.npy',
 'knnKlasyfikator_gruped_03.npy',
 'knnKlasyfikator_gruped_04.npy',
 'knnKlasyfikator_gruped_05.npy',
 'knnKlasyfikator_gruped_06.npy',
 'knnKlasyfikator_gruped_07.npy',
 'knnKlasyfikator_gruped_08.npy',
 'knnKlasyfikator_gruped_09.npy']

In [73]:
rf_dump = RandomForestClassifier(n_estimators=31, max_features=None, criterion='gini', max_depth=23)
rf_dump.fit(X,y)
joblib.dump(rf_dump, 'rfKlasyfikator')

['rfKlasyfikator',
 'rfKlasyfikator_01.npy',
 'rfKlasyfikator_02.npy',
 'rfKlasyfikator_03.npy',
 'rfKlasyfikator_04.npy',
 'rfKlasyfikator_05.npy',
 'rfKlasyfikator_06.npy',
 'rfKlasyfikator_07.npy',
 'rfKlasyfikator_08.npy',
 'rfKlasyfikator_09.npy',
 'rfKlasyfikator_10.npy',
 'rfKlasyfikator_11.npy',
 'rfKlasyfikator_12.npy',
 'rfKlasyfikator_13.npy',
 'rfKlasyfikator_14.npy',
 'rfKlasyfikator_15.npy',
 'rfKlasyfikator_16.npy',
 'rfKlasyfikator_17.npy',
 'rfKlasyfikator_18.npy',
 'rfKlasyfikator_19.npy',
 'rfKlasyfikator_20.npy',
 'rfKlasyfikator_21.npy',
 'rfKlasyfikator_22.npy',
 'rfKlasyfikator_23.npy',
 'rfKlasyfikator_24.npy',
 'rfKlasyfikator_25.npy',
 'rfKlasyfikator_26.npy',
 'rfKlasyfikator_27.npy',
 'rfKlasyfikator_28.npy',
 'rfKlasyfikator_29.npy',
 'rfKlasyfikator_30.npy',
 'rfKlasyfikator_31.npy',
 'rfKlasyfikator_32.npy',
 'rfKlasyfikator_33.npy',
 'rfKlasyfikator_34.npy',
 'rfKlasyfikator_35.npy',
 'rfKlasyfikator_36.npy',
 'rfKlasyfikator_37.npy',
 'rfKlasyfikator_38

In [74]:
rfG_dump = RandomForestClassifier(n_estimators=31, max_features=None, criterion='gini', max_depth=23)
rfG_dump.fit(X_gruped,y_gruped)
joblib.dump(rfG_dump, 'rf_Grupped_Label_Klasyfikator')

['rf_Grupped_Label_Klasyfikator',
 'rf_Grupped_Label_Klasyfikator_01.npy',
 'rf_Grupped_Label_Klasyfikator_02.npy',
 'rf_Grupped_Label_Klasyfikator_03.npy',
 'rf_Grupped_Label_Klasyfikator_04.npy',
 'rf_Grupped_Label_Klasyfikator_05.npy',
 'rf_Grupped_Label_Klasyfikator_06.npy',
 'rf_Grupped_Label_Klasyfikator_07.npy',
 'rf_Grupped_Label_Klasyfikator_08.npy',
 'rf_Grupped_Label_Klasyfikator_09.npy',
 'rf_Grupped_Label_Klasyfikator_10.npy',
 'rf_Grupped_Label_Klasyfikator_11.npy',
 'rf_Grupped_Label_Klasyfikator_12.npy',
 'rf_Grupped_Label_Klasyfikator_13.npy',
 'rf_Grupped_Label_Klasyfikator_14.npy',
 'rf_Grupped_Label_Klasyfikator_15.npy',
 'rf_Grupped_Label_Klasyfikator_16.npy',
 'rf_Grupped_Label_Klasyfikator_17.npy',
 'rf_Grupped_Label_Klasyfikator_18.npy',
 'rf_Grupped_Label_Klasyfikator_19.npy',
 'rf_Grupped_Label_Klasyfikator_20.npy',
 'rf_Grupped_Label_Klasyfikator_21.npy',
 'rf_Grupped_Label_Klasyfikator_22.npy',
 'rf_Grupped_Label_Klasyfikator_23.npy',
 'rf_Grupped_Label_Klas

## Zapisanie wyników klasyfikacji

In [76]:
knn_wynik = knn_dump.predict(X_test)

In [77]:
knn_wynik = le.inverse_transform(knn_wynik)

In [88]:
knn_wynik.tofile('knn_wynik.csv', sep=',', format='%s')

In [84]:
rf_wynik = rf_dump.predict(X_test)

In [85]:
rf_wynik = le.inverse_transform(rf_wynik)

In [89]:
rf_wynik.tofile('rf_wynik.csv', sep=',', format='%s')

In [90]:
rfG_wynik = rfG_dump.predict(X_test)

In [91]:
rfG_wynik = leG.inverse_transform(rfG_wynik)

In [92]:
rfG_wynik.tofile('rfG_wynik.csv', sep=',', format='%s')